# QADataset

In [2]:
%pip install langchain -q
%pip install pymupdf -q
%pip install langchain-community -q
%pip install langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.2 MB/s eta 0:00:00


In [ ]:
import fitz
import json
import glob
import os
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from resource.config import api_key

# 텍스트 추출
def extract_text_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.
    """
    docs = fitz.open(pdf_path)
    elements = []
    for page in docs:
        text = page.get_text()
        elements.append({'text': text.strip()})
    docs.close()
    return elements

# 데이터 구조
class QAPair(BaseModel):
    question: str = Field(description='Question generated from the text')
    answer: str = Field(description='Answer related to the Question')

# 프롬프트 템플릿
prompt = PromptTemplate.from_template(
    '''context information is below. You are only aware of this context and nothing else.
    {context}
    -----------------------------
    Given this context, generate questions based on the below query.
    You are a healthcare professional in {domain}.
    Your task is to provide exactly **{num_questions}** questions for an upcoming questions.
    You are not to provide more or less than this number of questions.
    The questions should be diverse in nature across the document.
    The purpose of questions is to use the information provided to inform people of first aid knowledge.
    You must also provide the answer to each question. The answer should be based on the context information provided only.

    Restrict the questions to the context information provided only.
    QUESTION and ANSWER should be written in Korean. Response in JSON format which contains the 'question' and 'answer'.
    ANSWER should be a complete sentence.

    # Format:
    ```json
    {{
        'QUESTION': '화상을 입었을 때는 어떻게 해야하나요?',
        'ANSWER': '화상부위를 흐르는 찬물 속에 넣어 적어도 10분 동안 담그고, 상처부위는 깨끗하고 가능하면 멸균 처리된 거즈로 덮습니다.'
    }},
    {{
        'QUESTION': '벌에 쏘였을 경우에는 어떻게 하나요?',
        'ANSWER': '피부에 벌침이 남아있는 경우, 신용카드 등으로 밀어서 제거한 후 2차 감염 방지를 위해 비누와 물로 씻어줍니다.'
    }},
    {{
        'QUESTION': '골절 사고를 당했을 때는 어떤 응급처치를 해야하나요?',
        'ANSWER': '심한 골절일 경우 부러진 뼈가 신경, 혈관, 근육을 손상하거나 복잡 골절이 되게 하는 일이 없도록 가능하면 부상자를 옮기지 말고 사고 발생 현장에서 움직이지 않도록 합니다.'
    }}
    ```
'''
)

# langchain 및 출력 파서 설정
parser = JsonOutputParser(pydantic_object=QAPair)

chain = (
    prompt
    | ChatOpenAI(
        model='gpt-4o',
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
        openai_api_key=api_key  # Replace with your API key
    )
    | parser
)

# QA 데이터 생성 및 저장
def create_qa_dataset_from_folder(folder_path, output_file):

    # 폴더 내 모든 PDF 파일 탐색
    pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))

    # QA 생성
    qa_pair = []
    for pdf_file in pdf_files:
        print(f"Processing file: {pdf_file}")
        elements = extract_text_pdf(pdf_file)

        for i, element in enumerate(elements):
            if element['text']:
                try:
                    qa_pair.append(
                        chain.invoke(
                            {'context': element['text'], 'domain': 'AI', 'num_questions': '3'}
                        )
                    )
                except Exception as e:
                    print(f"Error processing element in file {pdf_file}, page {i + 1}: {e}")

# 생성된 데이터 중간 출력
print("Generated QA Data:")


# 실행
if __name__ == '__main__':
    folder_path = '/content/drive/MyDrive/3rd project_pdf '  # PDF 파일이 저장된 폴더 경로
    output_file = 'qa_dataset.json'  # JSON 저장 파일 경로
    create_qa_dataset_from_folder(folder_path, output_file)  # 폴더 내 모든 PDF 파일을 처리

    # PDF 파일을 처리한 후
    pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))  # 폴더 내 PDF 파일 경로 리스트
    qa_pair = []

    for pdf_file in pdf_files:
        elements = extract_text_pdf(pdf_file)  # 각 PDF 파일에 대해 텍스트 추출

        # 각 페이지별 텍스트 처리 및 QA 생성
        for i, element in enumerate(elements):
            if element['text']:  # 텍스트가 존재하는 경우
                try:
                    qa_pair.append(
                        chain.invoke(
                            {'context': element['text'], 'domain': 'AI', 'num_questions': '3'}
                        )
                    )
                except Exception as e:
                    print(f'Error processing element {i+1} in file {pdf_file}: {e}')

    # 생성된 QA 데이터를 JSON 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(qa_pair, f, ensure_ascii=False, indent=4)
    print(f"QA dataset saved to {output_file}")



Generated QA Data:
Processing file: /content/drive/MyDrive/3rd project_pdf /[매뉴얼]_스포츠응급처치_매뉴얼.pdf
```json
{
    "QUESTION": "응급처치 매뉴얼이란 무엇인가요?",
    "ANSWER": "응급처치 매뉴얼은 응급 상황에서 적절한 조치를 취할 수 있도록 안내하는 지침서입니다."
},
{
    "QUESTION": "응급처치 매뉴얼의 주요 목적은 무엇인가요?",
    "ANSWER": "응급처치 매뉴얼의 주요 목적은 응급 상황에서 신속하고 효과적인 대응을 통해 부상자의 상태를 안정시키고 추가적인 손상을 방지하는 것입니다."
},
{
    "QUESTION": "응급처치 매뉴얼은 어떤 상황에서 사용되나요?",
    "ANSWER": "응급처치 매뉴얼은 다양한 응급 상황에서 사용되며, 예를 들어 화상, 골절, 벌에 쏘임 등의 상황에서 적절한 응급처치를 제공하는 데 사용됩니다."
}
``````json
{
    "QUESTION": "심정지 상황에서 심폐소생술은 어떻게 수행하나요?",
    "ANSWER": "심정지 상황에서는 즉시 심폐소생술을 시작해야 하며, 가슴 압박과 인공호흡을 번갈아 가며 수행합니다."
},
{
    "QUESTION": "골절 및 염좌가 발생했을 때 부목고정은 어떻게 하나요?",
    "ANSWER": "골절 및 염좌가 발생했을 때는 부목을 사용하여 손상 부위를 고정시켜 추가적인 손상을 방지합니다."
},
{
    "QUESTION": "출혈이 발생했을 때 지혈을 위해 어떤 조치를 취해야 하나요?",
    "ANSWER": "출혈이 발생했을 때는 깨끗한 천이나 붕대로 상처 부위를 압박하여 지혈을 시도합니다."
}
``````json
{
    "QUESTION": "열사병이 발생했을 때는 어떻게 해야 하나요?",
    "ANSWER": "열사병이 발생했을 때는 즉시 시원한 장소로 이동시키고, 옷을 느

In [8]:
# 데이터 형식 변형
fine_tune_data = []

for pair in qa_pair:
  if type(pair) == list:
    for qa in pair:
      message = {'messages' : []}
      system_message = {'role' : 'system'}
      user_message = {'role' : 'user'}
      ai_message = {'role' : 'assistant'}

      system_message['content'] = '이 시스템은 응급처치 절차를 설명해주는 시스템입니다.'
      user_message['content'] = qa['QUESTION']
      ai_message['content'] = qa['ANSWER']

      message['messages'].append(system_message)
      message['messages'].append(user_message)
      message['messages'].append(ai_message)

      fine_tune_data.append(message)

  elif type(pair) == dict:
    message = {'messages' : []}
    system_message = {'role' : 'system'}
    user_message = {'role' : 'user'}
    ai_message = {'role' : 'assistant'}

    system_message['content'] = '이 시스템은 응급처치 절차를 설명해주는 시스템입니다.'
    user_message['content'] = pair['QUESTION']
    ai_message['content'] = pair['ANSWER']

    message['messages'].append(system_message)
    message['messages'].append(user_message)
    message['messages'].append(ai_message)

    fine_tune_data.append(message)



# jsonl 파일 경로
file_path = 'qa_dataset.jsonl'

# jsonl 파일로 저장
with open(file_path, 'w', encoding='utf-8') as f:
    for item in fine_tune_data:
        # 각 딕셔너리를 JSON 문자열로 변환 후 한 줄씩 파일에 기록
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [9]:
fine_tune_data

[{'messages': [{'role': 'system', 'content': '이 시스템은 응급처치 절차를 설명해주는 시스템입니다.'},
   {'role': 'user', 'content': '응급처치 매뉴얼이란 무엇인가요?'},
   {'role': 'assistant',
    'content': '응급처치 매뉴얼은 응급 상황에서 적절한 조치를 취할 수 있도록 안내하는 지침서입니다.'}]},
 {'messages': [{'role': 'system', 'content': '이 시스템은 응급처치 절차를 설명해주는 시스템입니다.'},
   {'role': 'user', 'content': '심정지 상황에서 심폐소생술은 어떻게 수행하나요?'},
   {'role': 'assistant',
    'content': '심정지 상황에서는 즉시 심폐소생술을 시작해야 하며, 가슴 압박과 인공호흡을 번갈아 가며 수행합니다.'}]},
 {'messages': [{'role': 'system', 'content': '이 시스템은 응급처치 절차를 설명해주는 시스템입니다.'},
   {'role': 'user', 'content': '열사병이 발생했을 때는 어떻게 해야 하나요?'},
   {'role': 'assistant',
    'content': '열사병이 발생했을 때는 즉시 시원한 장소로 이동시키고, 옷을 느슨하게 하며, 체온을 낮추기 위해 물을 뿌리거나 젖은 수건으로 몸을 닦아줍니다.'}]},
 {'messages': [{'role': 'system', 'content': '이 시스템은 응급처치 절차를 설명해주는 시스템입니다.'},
   {'role': 'user', 'content': '응급처치의 기본 원칙은 무엇인가요?'},
   {'role': 'assistant',
    'content': '응급처치의 기본 원칙은 환자의 상태를 신속하게 평가하고, 필요한 경우 즉시 응급처치를 제공하여 생명을 구하고 상태를 안정시키는 것입니다.'}]},
 {'messag

In [ ]:
# 데이터 업로드
from openai import OpenAI

client = OpenAI(api_key=api_key)

response = client.files.create(
    file=open('qa_dataset.jsonl', 'rb'),
    purpose='fine-tune' # 데이터 파일의 목적 지정
)

file_id = response.id

In [ ]:
file_id

'file-9EbGvkJP6CjvYuBnjtEBE9'

In [11]:
# 모델 생성
client.fine_tuning.jobs.create(
    training_file=file_id, # 훈련할 데이터 세트 파일 ID
    model='gpt-4o-mini-2024-07-18', # 훈련시킬 모델 이름
    suffix= 'fine-tune-qadataset-model' # 훈련된 모델의 이름 지정
)

FineTuningJob(id='ftjob-AXkFLPhKr1sEk8CPo4b0Scpx', created_at=1732667439, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-q2w5bITBqBNCRpAkDRaoYxev', result_files=[], seed=1188015549, status='validating_files', trained_tokens=None, training_file='file-JuCBbB3mgAnqQSw2XEE1GG', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='fine-tune-qadataset-model')